This notebook focuses on extracting and tokenizing symptom text using google's gemini 1.5 flash --Sneha Dharne

In [1]:
import os
import re
import pandas as pd

In [23]:
import pandas as pd

df = pd.read_csv('symptoms_covid.csv')

In [24]:
df = df[0:10]

In [ ]:
YOUR_API_KEY = "***************************************"
                
import google.generativeai as genai

In [2]:
genai.configure(api_key=YOUR_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

try with first 100 data points first

In [9]:
document = """
,VAERS_ID,VAX_TYPE,SYMPTOM_TEXT,V_ADMINBY,FORM_VERS
0,2728994,COVID19,"muscle at the injection site is sore; He cannot lift anything more than 5 pounds.; Arm  soreness; This non-serious case was reported by a consumer via call center representative and described the occurrence of injection site muscle pain in a adult male patient who received RSVPreF3 adjuvanted (Arexvy) for prophylaxis.   Co-suspect products included Influenza vaccine for prophylaxis and Tozinameran (Pfizer BioNTech COVID-19 vaccine) for prophylaxis.   On an unknown date, the patient received Arexvy (right arm), Influenza vaccine (left arm) and Pfizer BioNTech COVID-19 vaccine (left arm). On an unknown date, an unknown time after receiving Arexvy, the patient experienced injection site muscle pain (Verbatim: muscle at the injection site is sore), activities of daily living impaired (Verbatim: He cannot lift anything more than 5 pounds.) and pain in arm (Verbatim: Arm  soreness). The outcome of the injection site muscle pain and activities of daily living impaired were not reported and the outcome of the pain in arm was unknown.   It was unknown if the reporter considered the injection site muscle pain, activities of daily living impaired and pain in arm to be related to Arexvy.     It was unknown if the company considered the injection site muscle pain, activities of daily living impaired and pain in arm to be related to Arexvy.   Additional Information: GSK receipt date: 22-DEC-2023 Reporter was the patient. The patient was received a dose of AREXVY 4 months ago on his right arm  and received a COVID-19 vaccine (Pfizer) and Flu vaccine (unknown manufacturer) on his left arm in the same visit.  The patient states that the muscle at the injection site was sore.  The patient cannot lift anything more than 5 pounds.  The patient was an X ray done on his rotator cuff to see if it was the cause for soreness.  The patient did not disclose results of exam. The reporter consented to follow up.",PHM,2
1,2729004,COVID19,"Micro-seizures starting approximately 2 weeks after 2nd dose given on 4/23/21. Seizures become more consistent resulting in a visit with primary care doctor and neurologist. An overnight stay was recommended on 06/23/21. Symptoms consisted of arrest in speech, eye rolling, arm stiffness. Micro-seizures happened about 5 times per hour and lasted for 5-8 seconds each time. Full seizure happened on 7/13/21, resulting in 4-day stay in the ICU. Pt. now takes 2 anti-seizure medications per day and 2 Rituximab infusions every 6 months (in-home). Lumbar puncture during ICU stay diagnosed him with the auto-immune disease called LGI1 Encephalitis.",OTH,2
2,2729006,COVID19,"On September 14, 2023, I was admitted to Hospital with what turned out to be Endocarditis.  The bacteria found from blood cultures was strep mitis.  I was in the hospital for one week and continued with 5 and 1/2 weeks of iv penicillin G.  When the course of antibiotics completed fever and chills came back.  At that time, 10/31/2023, I went to another Hospital and was admitted.  I was given more antibiotics and found that a spot on my lung had doubled in size.  It is believed that the spot was infection but we never determined the exact bacteria.  I was discharged on 11/8/2023 with a weeks worth of amoxicillin-clavulanate.  A follow up ct scan of lungs on 12/24/2023 showed the lung spot had resolved.",PHM,2
3,2729006,COVID19,"On September 14, 2023, I was admitted to Hospital with what turned out to be Endocarditis.  The bacteria found from blood cultures was strep mitis.  I was in the hospital for one week and continued with 5 and 1/2 weeks of iv penicillin G.  When the course of antibiotics completed fever and chills came back.  At that time, 10/31/2023, I went to another Hospital and was admitted.  I was given more antibiotics and found that a spot on my lung had doubled in size.  It is believed that the spot was infection but we never determined the exact bacteria.  I was discharged on 11/8/2023 with a weeks worth of amoxicillin-clavulanate.  A follow up ct scan of lungs on 12/24/2023 showed the lung spot had resolved.",PHM,2
4,2729006,COVID19,"On September 14, 2023, I was admitted to Hospital with what turned out to be Endocarditis.  The bacteria found from blood cultures was strep mitis.  I was in the hospital for one week and continued with 5 and 1/2 weeks of iv penicillin G.  When the course of antibiotics completed fever and chills came back.  At that time, 10/31/2023, I went to another Hospital and was admitted.  I was given more antibiotics and found that a spot on my lung had doubled in size.  It is believed that the spot was infection but we never determined the exact bacteria.  I was discharged on 11/8/2023 with a weeks worth of amoxicillin-clavulanate.  A follow up ct scan of lungs on 12/24/2023 showed the lung spot had resolved.",PHM,2
5,2729010,COVID19-2,"0300 12/30/23 Fever 102 degrees by oral thermometer, aches and pains, fatigue, chills, night sweats, diarrhea, loss of appetite.    Fever broke 0900 12/31/23 fatigue, diarhea, loss of appetite continued for another 36 hours",PHM,2
6,2729014,COVID19-2,patient's husband reported symptoms of prolonged shoulder pain.  patient was seen by provider (PCP) and urgent care with no resolution from providers.,PHM,2
7,2729015,COVID19,Patient started experiencing hives and was prescribed prednisone 10mg by her Doctor,PHM,2
8,2729016,COVID19,Have developed lumps on my thighs and wrist and arm,UNK,2
9,2729017,COVID19,"Lot number unclear on receipt.  Here is all info:  Spikevax 2023-24 Vial 50 mcg/0.5 ml inj.  Covid-19 (Sars-Cov-2)Mrna VA. Reaction was that my brain was in a fog for the next week.  Thinking was slowed, and took more effort than usual.  This is distinctly different from my reaction to prior shots (4 of them), which were generally flu symptoms for about 24hrs.  This was the first time my brain was affected.  After about a week I perceived going back to normal, but I do not know if there are any lasting effects since it's hard to compare exactly.",PHM,2
10,2729021,COVID19,"Patient is a 51 y.o. male patient of No, Physician with history of chronic respiratory failure s/p trach placement 8/17/2023, GERD, HTN  presented on 11/13/2023 with shortness of breath and fevers.  He was diagnosed with COVID.  HMS consulted by MD for transfer to intermediate.    He is discharged home in stable and improved condition on room air with supplies for tracheostomy care and referrals for ENT and urology.  Patient updated on discharge plan and agrees.   Acute on chronic respiratory failure?resolved Tracheostomy in place Status post trach placement 8/17/2023. Has a 5 Shiley extra long proximal cuffless On room air Patient not complaining of respiratory distress. Patient and his roommate educated on tracheostomy care Will need ENT follow up on discharge, referral sent   Sepsis, on admission - resolved COVID 19 Pneumonia Hx of multiple mdr infections Aspiration pneumonitis/pneumonia Initial ED vitals showed mild tachycardia, afebrile, hypotensive. VSS, AF Improved VS, on TM not in any distress, feeling much better. Blood cultures,Urine antigens, MRSA probe neg.Initial lactic acid 0.8 COVID swab positive, isolation precautions started Given recent hospitalization with history of multidrug-resistant organisms, ID started on broad spectrum antibiotics. Procalcitonin negative, sputum culture NGTD, resp PCR NGTD Stop emperic abx. Completed Remdesivir dosing On Decadron 6mg daily x10 days for COVID-19 Noted elevated AST/ALT, may be related to remdesivir, appears to have plateaued and are now downtrending Remove COVID precautions 11/24   Urinary incontinence, reported States started after stay in select and long-term Foley Postvoid negative for urinary retention, unlikely overflow incontinence Trial oxybutynin Outpatient referral for urology   Dysphagia Food noted in patient's trach tubing Speech recommends nectar thick liquids with puréed solids Continued SLP on discharge Patient educated again on diet   Hypomagnesemia Replacement ordered   HFpEF, without exacerbation Elevated Troponin BNP at 863, similar and reduced from prior Delta troponin flat, around patients baseline Echo 9/25/23: EF 60-65% Patient asymptomatic; no NTSEMI PEG tube Placed during prior hospitalization with tracheostomy Has not used PEG tube since discharge from nursing facility Tolerates p.o. intake Consulted general surgery for PEG tube removal.  Patient did not want to have PEG tube removed after hearing about the process.  Can follow-up about this outpatient. Info in AVS   Left shoulder pain States has been told he has rotator cuff tear but c/o recent trauma prior to admission and now with left shoulder pain.  Xray shoulder: Chronic separation of the left acromioclavicular joint, OP f/u    Anxiety Continue home meds   GERD Continue home meds",UNK,2
11,2729023,COVID19,"Woke up next morning with a lot of floaters in my right eye. Couldn?t get into see dr for 3 weeks, he said my retina was thin but okay. Saw another opthamalogist 6 weeks  later bc vision was impaired and he said I had a retinal tear in my eight eye and asked if I had any floaters recently and I said yes, at the end of May, the day after my covid vaccine. He said I?ve had this tear for a couple of months. That?s when I connected the dots between my vaccine and the retinal tear.  I had a detached retina in my left eye when u was 26 yo, so I am prone to retinal tears.",OTH,2
12,2729024,COVID19,"Patient is a 66 y.o. female patient of CNP with history of hypertension hyperlipidemia diabetes mellitus also is known for nonobstructive coronary artery disease with congestive heart failure nonischemic cardiomyopathy with ejection fraction 30 to 35% improved up to 62 according to last echo history of obstructive sleep apnea with morbid obesity is here due to chest pain epigastric pain and weakness noticed to have COVID-positive on arrival to the emergency department and hypoxic with oxygen saturation in the mid 80s improved with nasal cannula at 2 L and elevated blood pressure chest image showed infiltrates   COVID-19 respiratory infection Acute hypoxic respiratory failure Pneumonia due to COVID-19 Chest x-ray showed hazy airspace opacification at the right lung base which may represent pneumonia.  We will place patient on oxygen therapy and telemetry continue supportive measures check inflammatory markers infectious disease consult was placed, start the patient on Decadron and remdesivir  on room air and in no distress ;  Will follow labs with ID  Pt has had all immunizations prior to this Continue with Remdesivir and dexamethasone CT of chest with no sign of pneumonia 4mm right lower lung nodule is benign  Remdesivir completed Room air If ok with Dr will discharge home today   Chest pain - no complaints of - resolved Seem to be atypical and possible pleurisy related we will continue further trend of troponin levels and place patient telemetry  Troponin negative; CT chest 11-19 negative  Will order percocet prn for this complaint   Hypertension  resume home regimen Monitor the trend blood pressure was elevated on arrival, IV hydralazine as needed   Diabetes mellitus We will monitor blood glucose and insulin sliding scale check A1c level Nonischemic cardiomyopathy Chronic CHF with preserved EF known for nonobstructive coronary artery disease with congestive heart failure nonischemic cardiomyopathy with ejection fraction 30 to 35% improved up to 62 according to last echo    obstructive sleep apnea Resume CPAP while here   Morbid obesity  encourage healthier lifestyle",UNK,2
13,2729026,COVID19-2,"Patient is a 77 y.o. female patient of DO with a PMHx of atrial fibrillation (on apixaban), breast cancer s/p mastectomy (over 30 years ago), HTN, GERD, osteoporosis, and diet-controlled DM2 admitted for COVID19 pneumonia.      Acute hypoxic respiratory failure  COVID19 pneumonia Sepsis POA secondary to COVID with tachycardia, hypoxia Date of onset of symptoms: 11/21/23 Symptoms present on admission: SOB, fatigue, HA, coughing  Date of positive COVID19 test: 11/22/23  Imaging: CXR on admission negative for infiltrate --CTPA 11/22 neg for PE or infiltrate. Mild emphysema, bibasilar atelectasis noted. Oxygen requirements on admission: 2 L O2 on admit (none at baseline) Current oxygen requirements: RA Medical therapy: remdesivir, steroids,  all started 11/22  - DuoNebs q6h while awake Stopped ROcephin/azithromycin 11/23 as no evidence of secondary bacterial infection Consultants following: None Anticipated special isolation end date: 12/02/23 RA am 11/23, able to walk in room but still significant tachycardia and exertional dyspnea + coughing fits, added mucinex, tessalon Much improved, tachycardia resolved, RA, walking around room without issue, cogh better Will discharge with short course decadron only secondary to rapid improvement and worsening GERD, DM Nonspecfic LAD on CT - has regular follow up with Dr with imaging   PAF Remote DVT Continued home diltiazem and apixaban No evidence of RVR on admission Sinus tachycardia 11/23 , treat COVID Resolved, refill for cardizem ordered     Essential HTN HLD Continued home lisinopril Continued home atorvastatin   Diet controlled DM2 Hgb A1c 9/9/23 - 6.6% Carb consistent diet  Sensitive sliding scale insulin ordered given risk of hyperglycemia while on steroids BG reasonable to date Short course steriods due to hyperglycemia and should resolve when off steriods   GERD Takes prilosec OTC and will take BID at home until off steriods Does not need script",UNK,2
14,2729030,COVID19,"Migraine, nausea, tachycardia, L leg pain and pulling (front of thigh, back of calf), heaviness of heart, SOB, irritability, fatigue,",PVT,2
15,2729031,COVID19,Had a massive heart attack on 03/25/2022 100% blockage in LAD,PVT,2
16,2729031,COVID19,Had a massive heart attack on 03/25/2022 100% blockage in LAD,PVT,2
17,2729034,COVID19,"2-3 weeks after receiving the vaccine, developed dry mouth, chapped lips and irritation when eating (even non-spicy foods).  Had poor hearing before vaccine and hearing loss got worse.  loss of clarity in hearing words.",PHM,2
18,2729035,COVID19,"Redness, itchiness, skin hot to touch, left shoulder and bicep swelling by 7am the following day, Saturday.  By 7am Sunday, under arm area on the left side was swollen.",UNK,2
19,2729036,COVID19,"A family of six came in for COVID vaccines.  The father came in a few minutes late requesting COVID and flu vaccines.    After preparation of vaccines, the family was brought into exam room.  I was accompanied by another nurse to assist if needed.  The identity of each family member was validated, and other pertinent information reviewed.  The COVID vaccine was administered to one of the children, after which, the parents requested flu vaccines for the rest of the family.  The other nurse went to prepare the vaccines at my request.  Immediately after administration of the COVID vaccine to pt. it was discovered the he was inadvertently given the pediatric dose of the COVID vaccine.  There were no symptoms and no treatment needed.",PUB,2
20,2729039,COVID19,"Redness, swelling, soreness at injection site and swollen lymph nodes in armpit.",PHM,2
21,2729048,COVID19,"viral sore; Covid-19; This spontaneous case was reported by a patient and describes the occurrence of COVID-19 (Covid-19) and VIRAL PHARYNGITIS (viral sore) in a  patient of an unknown age and gender who received SPIKEVAX NOS (SPIKEVAX NOS) for COVID-19 prophylaxis.     No Medical History information was reported.    On an unknown date, the patient received dose of SPIKEVAX NOS (SPIKEVAX NOS) (unknown route) 1 dosage form. In December 2023, the patient experienced COVID-19 (Covid-19). On 25-Dec-2023, the patient experienced VIRAL PHARYNGITIS (viral sore). At the time of the report, COVID-19 (Covid-19) and VIRAL PHARYNGITIS (viral sore) outcome was unknown.        The action taken with SPIKEVAX NOS (SPIKEVAX NOS) (Unknown) was unknown.       No concomitant medication was reported.  It was reported that patient had all of the Moderna vaccines The initial ones and the boosters and now patient had, actually, after three years patient had covid. But last night patient developed a viral sore, it was not raves but it's on the top of pallet on top of the teeth. No treatment medication was reported.   Reporter did not allow further contact",UNK,2
26,2729136,COVID19,"COMIRNATY VACCINE (PLASTIC TUBE) WAS ADMINISTERED ON 12/23/2023. THE VACCINE WAS RECEIVED ON 10/12/2023, THE 10 WEEKS WAS ON 12-20-2023, SO THE VACCINE WAS ADMINISTERED THREE DAYS AFTER THE 10 WEEKS. PATIENT WAS CONTACTED AND THE SITUATION WAS EXPLAINED. WE RECOMMENDED TO WAS CONTACT HER DOCTOR WHO KNOWS HER HEALTH PROFILE AND CAN DO A BETTER SUGGESTION IN REGARDS REVACCINATION OR NOT.                                                                                  SHE REPORTED ARM AND LOWER BACK PAIN FOR ONE DAY.",PHM,2
27,2729137,COVID19,"COMIRNATY VACCINE (PLASTIC TUBE) WAS ADMINISTERED ON 12/23/2023. THE VACCINE WAS RECEIVED ON 10/12/2023, THE 10 WEEKS WAS ON 12-20-2023, SO THE VACCINE WAS ADMINISTERED THREE DAYS AFTER THE 10 WEEKS. PATIENT WAS CONTACTED AND THE SITUATION WAS EXPLAINED. WE RECOMMENDED TO CONTACT HIS DOCTOR WHO KNOWS HIM HEALTH PROFILE AND CAN DO A BETTER SUGGESTION IN REGARDS REVACCINATION OR NOT.                                                                                  HE REPORTED ARM PAIN.",PHM,2
28,2729138,COVID19,"COMIRNATY VACCINE (PLASTIC TUBE) WAS ADMINISTERED ON 12/24/2023. THE VACCINE WAS RECEIVED AT THE PHARMACY ON 10/12/2023, THE 10 WEEKS FOR EXPIRATION WAS ON 12-20-2023, SO THE VACCINE WAS ADMINISTERED FOUR DAYS AFTER THE 10 WEEKS. PATIENT WAS CONTACTED AND THE SITUATION WAS EXPLAINED. WE RECOMMENDED TO CONTACT HER DOCTOR WHO KNOWS HER HEALTH PROFILE AND CAN DO A BETTER SUGGESTION IN REGARDS REVACCINATION OR NOT.  SHE DIDN'T REPORTED ANY SIDE EFFECTS OR ADVERSE EVENT.",PHM,2
29,2729139,COVID19,"Admit date: 12/11/2023   Discharge date: 12/22/2023   Admitting Physician: MD    Discharge Physician: MD   Reason for Admission: COVID plus community-acquired pneumonia   Hospital Course:   Per note of DO: ""Patient is a 86-year-old male with a past medical history of hypertension, type 2 diabetes mellitus prostate cancer with bone metastasis who presents with shortness of breath.  Found to have COVID plus community-acquired pneumonia.  Due to generalized worsening antibiotics broad to cefepime but was still having fevers and encephalopathy.  Cefepime switched to Zosyn to see if this was contributing to encephalopathy. Fever resolved and AMS improving with switch to Zosyn,  Lumbar puncture negative for meningitis.  Neurology did not recommend any other testing.  Possibly just COVID encephalitis vs hospital delirium. ""   Reportedly patient was found in bed cold and unresponsive about 45 minutes after being checked previously. CODE BLUE was called and CPR  was initiated. Cardiac rhythm of asystole did not respond to three rounds of epinephrine. Time of death pronounced at 0525.  Attempted to call wife; call directly went to voicemail; message left requesting wife to call the hospital.    Procedures:  MRI brain negative for acute illness LP not suggestive of meningitis   Consults: Neurology   Discharge Problem List      * (Principal) Acute respiratory insufficiency   Type 2 diabetes mellitus with hyperglycemia, without long-term current use of insulin   COVID   CAP (community acquired pneumonia)   Acute encephalopathy     Wt Readings from Last 1 Encounters: 12/21/23 82.1 kg (180 lb 14.4 oz)   Discharged Condition: Deceased",PVT,2
30,2729142,COVID19,"DO Last attending   Treatment team Acute psychosis   Discharge Summary DO (Resident)   Emergency Medicine Cosigned by: DO at 12/25/2023  9:17 AM  Attestation signed by, DO at 12/25/2023  9:17 AM  I was physically present for the key or critical portions of the Evaluation and Management services performed by the resident. I personally evaluated and examined the patient. I participated and discussed the case with the resident and agree with the findings and the plan as documented in the resident's note.   Discharge Summary  Patient here for acute psychosis with dangerous delusions.  Patient will be admitted to the hospitalist service for further evaluation with psychiatry on consult.Dr. will be on consult. Left Against Medical Advice",PVT,2
31,2729144,COVID19,"Pt started feeling dizzy immediately after vaccine was given. BP 160/80. Diaphoretic. After 1 hour, pt was continuing to have the same side effects. Doctor referred to ER. Pt given BP medication at ER and sent home. Pt still feeling dizzy 4 days later.",PUB,2
32,2729145,COVID19,"Admit date: 12/19/2023  Discharge date: 12/26/2023   Admitting Physician: MD   Discharge Physician: MD  Reason for Admission: syncope   Discharge Problem List     Acute cystitis without hematuria   Syncope   COVID-19 virus infection   Epilepsy   Hypothyroidism   Chronic back pain   CKD stage G3b/A1, GFR 30-44 and albumin creatinine ratio <30 mg/g (*)   Gout   Normocytic anemia   Type 2 diabetes mellitus, without long-term current use of insulin (HCC)  Procedures: None.    Consultations: None   Hospital Course:  Patient had been feeling poorly for a week and reportedly had an episode of Syncope and had a ground level fall but apparently no significant injuries, some bruising.  Syncope may have been from volume loss with poor oral intake and diarrhea that led to acute on chronic kidney injury with background of CKD stage G3b/A1, GFR 30-44.  Improved with hydration and no further syncope.  Found to have  COVID-19 virus infection but has not been hypoxic so was not started on targeted therapy.  Noted to have Acute cystitis without hematuria with culture growing. E.coli.   Received three days of CEFTRIAXONE  Epilepsy LEVETIRACETAM  Hypothyroidism LEVOTHYROXINE Gout  ALLOPURINOL no active disease  Chronic back pain chronic pain medication  Type 2 diabetes mellitus, without long-term current use of insulin on sliding scale  Normocytic anemia likely of chronic disease. Discharge disposition: Rehab Facility / Unit",PVT,2
34,2729148,COVID19,Flu-like symptoms. Platelet crash.,PHM,2
37,2729153,COVID19,"After 1 hour - developed 'shaking' throughout body like pt was freezing to death but was not cold.  later developed into internal shaking lasting all day saturday 12/30. Went to urgent care Monday 1/1/24  * patient also reported back pain, mental deficits where saying wrong words or forgetfulness of details",PHM,2
40,2729160,COVID19-2,"Admit date: 12/25/2023  Discharge date: 12/26/2023  Admitting Physician: MD   Discharge Physician: MD  Admission Diagnoses: Hypoxia [R09.02] Pneumonia due to COVID-19 virus [U07.1, J12.82] Acute respiratory insufficiency [R06.89]  Discharge Diagnoses: Principal Problem:   Acute respiratory insufficiency Active Problems:   COVID-19 virus infection   Type 2 diabetes mellitus with cardiac complication    Essential hypertension, benign   Mixed hyperlipidemia   Coronary artery disease of native artery of native heart with stable angina pectoris    Parkinson's disease   Artificial pacemaker   Compulsive behavior disorder    Mild episode of recurrent major depressive disorder    Chronic combined systolic and diastolic heart failure    Thrombocytosis   Hyperbilirubinemia   Hypoxia   Fever  Admission Condition: poor  Discharged Condition: good  Indication for Admission: Hypoxemia  Hospital Course: Elderly man with Parkinson's disease presents with malaise cough and generalized weakness and on ER evaluation found to have mild hypoxemia and fever and COVID positive.  Patient corrected with NC oxygen and was admitted and treated with steroids and oxygen.   He rapidly improved and weaned to room air.  He improved and requested discharge to home.  CXR was unremarkable.  Referred to home care for follow up of COVID and hypoxemia.",PVT,2
42,2729163,COVID19,"Admit date: 12/22/2023  Discharge date: 12/26/2023   Admitting Physician:  MD   Attending Physician at the Time of Discharge:  DO  Primary Care Physician: No primary care provider on file., None  Reason for Admission:  Acute hypoxic respiratory failure in the setting of COVID-19 infection  Hospital Course: 79-year-old female with past medical history significant for hypertension, hyperlipidemia, anxiety, diabetes mellitus type 2 with long term insulin use, chronic pain with long-term opioid use who presented to  emergency department on 12/22 with worsening symptoms of debility, nausea, vomiting, shortness of breath in the setting of COVID-19 infection.  During the admission patient completed a short course of remdesivir, she was also started on dexamethasone.  Due to the hypoxic respiratory failure patient required oxygen supplementation throughout most of her stay, up to 2 L oxygen via nasal cannula.  On the day of discharge, patient was able to be weaned off oxygen.  Ambulation test revealed patient is saturating well at 94% on room air.  Symptoms improved significantly prior to discharge, she is being discharged in 12/26 in stable condition. Discharged to home.",PVT,2
43,2729165,COVID19,"Admit date: 12/23/2023  Discharge date: 12/27/2023  Admitting Physician: MD   Attending Physician at the Time of Discharge: MD  Primary Care Physician: DO FACOI Reason for Admission: Viral sepsis, neutropenic fever  Hospital Course: Patient is a 78-year-old male with a past medical history of lung cancer receiving chemotherapy with Dr. paroxysmal atrial fibrillation, hypertension, hyperlipidemia, sick sinus syndrome status post pacemaker, presented to the emergency department for a syncopal episode.  Patient was admitted for viral sepsis due to COVID-19 and likely postural hypotension and dizziness due to initiation terazosin at 5 mg twice daily by primary care provider for blood pressure management.  Patient was placed on cefepime and remdesivir protocol.  Patient did not have any evidence of hypoxia.  Patient was also found to be neutropenic which resolved with granulocyte colony stimulating factor.  During hospital course, patient was also found to have worsening thrombocytopenia where in 1 unit of platelets was transfused with improvement.  Patient will need to follow up with heme Onc for pancytopenia and continued treatments for lung cancer.  Patient will also need to follow up with primary care provider for blood pressure management.  Blood pressure is inpatient were acceptable, advised patient to keep a blood pressure log and if an additional agent is needed, we do suggest starting an alternative medication other than terazosin such as amlodipine.  Discharged on Augmentin and ciprofloxacin to complete course for neutropenic fever. Discharged to home.",PVT,2
45,2729167,COVID19,"Admit date: 12/27/2023  Discharge date: 12/29/2023     Reason for Admission: Coughing, SOB  Brief Summary of Hospital Stay: PT. is a 63 YO female who presents with cough, SOB. She has relevant medical history of  dysphagia, tobacco abuse, chronic obstructive pulmonary disease (?), schizophrenia.   On presentation patient stable with symptoms of coughing ""until blue in the face"" and/or until she threw up. Tested positive for COVID on admission, blood cultures negative. CXR with R hemidiaphragm elevation, R lung base strandy opacifications. Mild leukocytosis on labs. Treated with steroids, remdesivir and symptomatic management. Oxygen weaned on 12/29 with normal walk test. Discharged in stable condition back to facility with steroid course, inhalers, symptomatic   Discharged to home.",PVT,2
46,2729169,COVID19,I mistakenly gave the client a Twinrix vaccine in her LD when she was immunized as a child for Hepatitis A&B. The client's husband was present at the visit and was supposed to receive that vaccine. The client was informed of the mistake.,PUB,2
48,2729172,COVID19,"She got her vaccine, and Saturday afternoon she started having a cough from her chest, not a sore throat.  It got progressively more frequent Saturday night, but definitely out of the ordinary and deep.  Sunday, she felt generally OK, fatigued but she worked in her yard and did some other things.  By Sunday night she was starting to get sick, coughing was almost like bronchitis and really deep.  She started getting a head cold situation, and just generally feeling bad, joints hurting, back hurting and her hips as well.  Her arm where the injection was sore, but she expected that.  By Sunday night she was just full time sick and could not move.  She was in bed all day yesterday as it hurt to get out of bed, did not e anything other than a little biscuit.  The coughing hurt, she had a fever this whole time that broke Sunday night and then came back yesterday.  She did not measure the temp, but had chills, flu-like symptoms the entire time.  She has been taking Nyquil for her symptoms.  She started to get a headache yesterday afternoon and would take the Nyquil and it would go away, and then woke up with an excruciating headache and took some Advil for it.    She has not called her doctor.  She is feeling better today, not going to work today but feeling better than she did yesteday and able to move around some.",PHM,2
49,2729174,COVID19,"Admit date: 12/29/2023  Discharge date: 12/30/2023    Reason for Admission: Acute hypoxic respiratory failure due to COVID  Brief Summary of Hospital Stay: 85yoF hx below presented for SOB, cough, generalized myalgia/chills, COVID+, hypoxic and dyspneic. Admitted for O2 therapy and decadron, symptoms reported to start 12/14/23 and likely has post viral bronchitis and not still virulent with covid, oxygen rapidly weaned to RA and she ambulated around the room for me with a walker without issues, requested discharge home. Will not continue steroids given no wheeze or hypoxia on discharge. Spoke to her daughter that given CDC guidelines she is no longer infectious given >10d from symptoms onset. Discharged to home.",PVT,2
50,2729177,COVID19,"Within 4 hours my heart converted from normal sinus rhythm (NSR) to atrial fibrillation (AF).  It remained there for 2 weeks until I had a cardioversion.  For the next month it was mostly NSR, with 4 episodes of AF lasting less than 24 hr.  After that it converted to AF and stayed there for over 3 months until I had an ablation 12/15/23.  That converted it to NSR for 48 hours and I am back in AF again.  Prior to the vaccine, my AF was only intermittent and was fairly well controlled by controlling triggers.  After the vaccine, it converted to constant AF without a break.",PHM,2
51,2729179,COVID19,"Discharge Examination & Summary  Admission Date and Time: 12/19/2023  6:46 PM Discharge Date: 12/31/2023 Admission Diagnosis: Mild traumatic brain injury Discharge Diagnosis: Mild traumatic brain injury  Diagnosis ? Mass of uterine cervix ? Anemia ? Depression ? Invasive squamous cell cancer of the cervix ? Malignant neoplasm of endocervix ? Gallstones ? Mild traumatic brain injury ? SAH (subarachnoid hemorrhage) ? SDH (subdural hematoma) ? Orbit fracture, right ? Closed fracture of distal phalanx of right little finger ? Anxiety ? Constipation ? Vitamin D deficiency ? Scalp laceration ? Fracture of distal end of right humerus ? Tibia/fibula fracture, right, closed, initial encounter ? Major traumatic injury                                                    PREMORBID FUNCTIONAL STATUS Premorbid Functional Status:  Level of Independence: Independent with transfers; Independent with ambulation    REHABILITATION COURSE  Pt. is a pleasant 50 YO right handed female with anxiety who presented 12/19/2023  6:46 PM to hospital after primary admission to  Hospital for major multiple trauma after MVC.  On 12/12/23, patient remembers that she was driving and then hitting a car passing perpendicularly in front of her on the side of their car. Then her car was hit from behind and then she blacked out. She remembers the EMTs but doesn't remember if in or out of ambulance. Then she remembers waking up again when they were explaining that she was being taken to hospital and then woke up again at hospital.  List of injuries as follows:  Right arm (distal humerus) fracture Right leg, foot (tibia, fibula) fractures Scalp lacerations Subarachnoid hemorrhage  Subdural hemorrhage  Right periorbital contusion and blowout fracture  Right fifth distal phalanx fracture  She underwent the following procedures: 12/12: Repair of scalp lacerations  12/12: Reduction of right lower extremity fracture by Orthopedic surgery 12/12: Wrist splinting of right upper extremity by Orthopedic surgery 12/12 Left great toe washout and closure  12/13: ORIF R Ankle fracture, pinning 1st and 2nd toes 12/14 - ORIF R distal humerus fracture 12/15 - ORIF R bimall ankle fx, R syndesmotic repair, gissane spike for ankle dislocation, ORIF 1st and 2nd proximal phalanx, open reduction without fixation of 4th proximal phalanx, I&D R open 1st, 2nd, 4th proximal phalanx, I&D R knee traumatic arthrotomy, closed tx w/o manipulation of R calcaneus anterior process fx, R short leg splint application   Hospital course was complicated by incidental cholelithiasis which patient may follow up with PCP outpatient. She was also seen by SLP and neuropsych who found her demonstrating good cognitive recovery  When medically stable, the patient was discharged for acute inpatient rehabilitation of functional and cognitive deficits.  Symptoms related to these abovementioned conditions and events were well-controlled, and the patient was discharged medically stable and improved. There were no significant acute episodes that affected length of stay or tolerance to therapies.   The patient was also provided orthotics / durable medical equipment to ensure improved function and safety in the rehabilitation and home setting.  Occupational, Physical and Speech Language Pathology therapists and a Neuropsychologist were consulted for evaluation and treatment so as to best determine the amount and extent of services required for cognitive and functional optimization. The patient received a minimum of 180 minutes of therapy per day for 5 out of 7 days, or more than 15 hours of intensive therapy over the course of a week.   While in the unit, the acute rehabilitation service also held interdisciplinary team conferences on a weekly basis to establish specific short- and long-term goals during rehabilitation, and for discharge. Barriers to discharge were also noted and updated as they were addressed. Plans to assure continuity and coordination of care after discharge from the acute rehabilitation unit were also established and explained to the patient and care provider(s). Discharged to home.",PVT,2
52,2729187,COVID19,VACCINE GIVEN AFTER VIABILITY DATE.,PVT,2
53,2729192,COVID19,"Left arm sore after shot, pain never went away. affecting mobility, very concerned. possibly contracted sirva(patient recognizes symptoms while looking it up online). pain on job site, just came from ER. Took tramadol and cyclobenzaprine this AM 01-02-2024.",PHM,2
54,2729194,COVID19,"General malaise, subjective fevers/chills, congestion, rhinorrhea, cough x2 days.",PVT,2
55,2729194,COVID19,"General malaise, subjective fevers/chills, congestion, rhinorrhea, cough x2 days.",PVT,2
56,2729195,COVID19-2,Spontaneous bruising noted to bil UEs on 12/31/23.  Bruising worsened today.  When resident was asked about it he stated he was unaware how they occurred.  Did state that nothing had happened like bumping anything.  Bruising is inconsistent and scattered.,SEN,2
57,2729197,COVID19,"She got her vaccine, it didn't hurt getting it, but she has had reactions whenever she had her first COVID vaccine and everyone she has had from it but nothing like this one.  The others were tolerable.  This was very painful, her muscles just ached.  She had been going to PT to strengthen her hip muscles, and those muscles were so painful after the injection that she could not move when she was in bed as when she moved she would hurt all over.  It was so scary as she felt she would get some reaction but did not know it would be so hurtful.  Her temperature was 100 degrees, and she felt a little bit on the dizzy side and very tired, which she had not had the reaction with the other vaccines.  Instead of lasted 2 day it lasted 4 days before it left her body.  She did not take anything for her symptoms as she just wanted to see how it played out without taking anything.  She stayed in bed for the first day and a half and did not eat much other than some liquid and some small pieces of fruit, and then started slowly and then by the 4th day she was cooking.  She is now feeling much better.  She felt it was unusual for her to get this as she had not had this reaction before.",PHM,2
58,2729198,COVID19,Vaccine administered was past 10 week beyond use date. BUD of 12/11/2023. Patient denies any adverse effects.,PHM,2
59,2729200,COVID19,Vaccine administered past 10 week beyond use date. BUD of 12/11/2023.,PHM,2
60,2729201,COVID19-2,"Location-  locatized where he had shots, elbows and both thumbs.    Duration- acording to what he dose Characteristics -feels like his arm feel asleep.   started when he had COVID 19 and pneu shot.   If he streches his arms out it hurt, he has been having arm  Aggravating Factors- lifting, when he lays on it. Cold bothers the most, stretching out straight  Treatment- tiger balm helps, warm helps cold makes it worse",PVT,2
61,2729203,COVID19,Vaccine was administered past 10 week BUD. BUD of 12/11/2023. Attempted to reach patient but unable to reach. Unknown if patient has experienced any adverse effects.,PHM,2
62,2729204,COVID19,No adverse reactions.  Individual is 9 years old and was given dose for 12yrs and above.,PUB,2
63,2729211,COVID19,"He got his vaccine, he didn't notice anything at first, but about 6 months after that he started to feel some bones in his right arm.  He looked at his shoulder and his shoulder was on bone.  His flesh and muscle has all eroded from his right arm.  It is moving to his right leg and to his left side.  He has been to numerous physicians who told him that he has had no problems.  He has been going to the doctor nonstop and taking steps for about 10 months, and he says that it is slowly taking away his flesh.  The best way to describe it is his body is slowly eating itself asymmetrical from right to left.  He has been to a neurologist, Rheumatologist, Oncologist, Nephrologist who have not found anything in regard to  what is going on.  Has been to 30+ doctors, and has had blood work, MRI's and muscle tests and they cannot find anything wrong with him.  They do not recognize it as they have not seen him in the past, but he is loosing strength in his right side and is right handed dominent and his left arm is now stronger than his right now.  He has had his eyes on it for the last 10 months and has slowly seen his body eat itself.  He had a nerve conduction study done the other day that did not show any problems with his muscles.  He feels that there is something is going on, and he can see the radius in his arm and veins in his arm that he could not see a year ago and feels that his body is slowly dying.  He is a healthy 50 year old guy and feels that in a year or so he will be noticed and then it will be too late to do anything for him.  The doctors have not given any diagnosis or any answers to his problem.    He noticed it first with his wrist and then he felt his bones and then felt his shoulder and said it's just like bone, muscle and tendon's.  Tomorrow he is having a CAT scan of his pancreas to see if there is something to do with that.  The worst spot is his right shoulder, right arm and felt that he may have had ALS and they tell him that his muscles are fine, but he said that if he had to lift weights that he would not be able to and his strength is slowly eroding in his entire body.  He says that the connective flesh is gone.  He was informed that they felt that he has anxiety, but all the doctors are making him feel like he is making up things.  He never went to the doctor in the past as he was healthy and he feels that they feel he is making things up.    If he goes to the ER they tell him that there is no emergency.  He has been sent away from the ER a  couple of times as they send him away.  His blood work is all OK.  Says that his arm looks like somebody took a chunk out of his arm.  His left arm is now doing the same thing but slower.   He is starting to get pain in his right shoulder and right wrist area as the connecting flesh is eroding away and has not experienced this pain before.  These are the two weakest areas on his body.    By the time he had the 4th vaccine his symptoms were full blown and he says that his arm is slowly deconstructing itself, and his arm is literally caving in.  LIke his flesh is melting off of the bone and it's terrifying.",PHM,2
64,2729221,COVID19,Vaccine expired by 1 day (refrigerated),UNK,2
65,2729222,COVID19,Pt received the covid booster on 12/30/23 and next day pt developed left side mouth drooping on outside/inside his mouth and difficulty with drinking/speech. Symptoms are present now and have been constant since 12/31/23.  Pt states he has had bells palsy before.,PVT,2
66,2729224,COVID19-2,hospitalized with COVID,PVT,2
67,2729226,COVID19,hospitalized with COVID,PVT,2
68,2729232,COVID19,"patient with dizziness, nauseous, felt like room was closing in, syncopal.  Paramedics called and evaluated patient, symptoms improved",PHM,2
69,2729234,COVID19,hospitalized with COVID,PVT,2
70,2729240,COVID19,hospitalized with COVID,PVT,2
71,2729244,COVID19-2,hospitalized with COVID,PVT,2
72,2729246,COVID19,hospitalized with COVID,PVT,2
73,2729253,COVID19,Updated COVID-19 vaccine (2023-2024) indicated for patients 6mo-11yr was given to this 37 year-old patient. Nurse confirms the patient received a full 0.5 mL dose.,UNK,2
74,2729254,COVID19,Spikevax 2023-2024 Formula.  Patient had no symptoms during the time of administration or until discharge. Patient is not aware of event yet.,PVT,2
75,2729257,COVID19,"Spikevax 2023-2024 Formula. No known adverse reactions, patient is not aware of event.",PVT,2
76,2729258,COVID19,Death after recent Covid illness,PUB,2
77,2729259,COVID19,No adverse effects at the time of administration and 15 minutes after as patient did receive other vaccinations..,PVT,2
78,2729260,COVID19,pain in neck,UNK,2
79,2729262,COVID19,"Approx 2 weeks after COVID shot, and 1 week after flu shot, noticed painless, blurred vision in right eye.  Visual field constriction,",PHM,2
80,2729263,COVID19-2,My menstrual period has been over two weeks late since getting the vaccine. I am 100% not pregnant. Late/missing periods are not normal for me.,UNK,2
81,2729325,COVID19,"Neuropathy; stage 1 chronic kidney disease; tingling in feet/vibrations in feet; severe muscle pain in her arms and legs; This serious case was reported by a consumer via call center representative and described the occurrence of neuropathy in a 64-year-old female patient who received Herpes zoster (Shingrix) for prophylaxis.   Co-suspect products included COVID-19 vaccine for prophylaxis.   Concurrent medical conditions included thyroid disorder (it was not clarified or specified), chronic lymphocytic leukemia, osteoporosis and osteoarthritis. Additional patient notes included patient did not had shingles in the past and not had diabetes.   In MAR-2021, the patient received the 1st dose of Shingrix. On an unknown date, the patient received COVID-19 vaccine. On an unknown date, several days after receiving Shingrix, the patient experienced neuropathy (Verbatim: Neuropathy) (serious criteria GSK medically significant), chronic kidney disease stage 1 (Verbatim: stage 1 chronic kidney disease) (serious criteria GSK medically significant), tingling of extremity (Verbatim: tingling in feet/vibrations in feet) and myalgia upper extremities (Verbatim: severe muscle pain in her arms and legs). The outcome of the neuropathy and tingling of extremity were not resolved and the outcome of the chronic kidney disease stage 1 was unknown and the outcome of the myalgia upper extremities was resolving.   It was unknown if the reporter considered the neuropathy, chronic kidney disease stage 1, tingling of extremity and myalgia upper extremities to be related to Shingrix.     It was unknown if the company considered the neuropathy, chronic kidney disease stage 1, tingling of extremity and myalgia upper extremities to be related to Shingrix.   Additional Information: GSK Receipt Date: 24-DEC-2023 and 29-DEC-2023 The patient self-reported this case. The patient received the first dose of Shingrix in mid-March 2021 and Within a few days, the patient began experiencing severe muscle pain in her arms and legs, tingling and vibrations in her feet.  the next several weeks had severe muscle pain, it felt like someone was pulling the muscles off my bone. The tingling in feet also started and had continued since. The reporter asked if this tingling in feet would stay forever. The patient had the test for neuropathy, and it was confirmed. The patient was scared and was terrified of getting shingles, but now this on top of everything else she had to deal with too much. The reporter asked if the symptoms would get worse. The patient stated that if she had known this would happen, she would had never gotten the vaccine. The patient was diagnosed with neuropathy. The patient reported that Stage 1 chronic kidney disease developed after receiving Shingrix. The patient stated that she received a COVID vaccine around the same time as the first dose of Shingrix. The patient reported that the pain in her muscles were much better, but the tingling in her feet remained, some days it was minimal and other days it felt like she had a tens unit on her feet. The patient did receive the second dose of Shingrix, but the described events started after the first dose. The reporter consented to follow up. On an unknown date, several days after receiving COVID-19 vaccine, the patient experienced neuropathy, chronic kidney disease stage 1, tingling of extremity and myalgia upper extremities. It was unknown if the reporter considered the neuropathy, chronic kidney disease stage 1, tingling of extremity and myalgia upper extremities to be related to Covid 19 vaccine.",UNK,2
82,2729333,COVID19,"Covid-19; This spontaneous case was reported by a patient and describes the occurrence of COVID-19 (Covid-19) in a 66-year-old female patient who received mRNA-1273 (Moderna COVID-19 Vaccine) (batch nos. 048A21A and 017F21A) for COVID-19 prophylaxis.     Concurrent medical conditions included Penicillin allergy, Blood pressure high (Since vaccine) since 2012, Non-sustained ventricular tachycardia since 17-Oct-2023, Dizziness since 17-Oct-2023 and Aneurysm (2 aneurysms) since 17-Oct-2023.   On 17-Feb-2021, the patient received first dose of mRNA-1273 (Moderna COVID-19 Vaccine) (unknown route) 1 dosage form. On 17-Mar-2021, received second dose of mRNA-1273 (Moderna COVID-19 Vaccine) (unknown route) dosage was changed to 1 dosage form. On 24-Oct-2021, received third dose of mRNA-1273 (Moderna COVID-19 Vaccine) (unknown route) dosage was changed to 1 dosage form. On 01-Sep-2022,  after starting mRNA-1273 (Moderna COVID-19 Vaccine), the patient experienced COVID-19 (Covid-19). At the time of the report, COVID-19 (Covid-19) outcome was unknown.      DIAGNOSTIC RESULTS (normal ranges are provided in parenthesis if available): On 01-Sep-2022, SARS-CoV-2 test: Positive.         No concomitant medications were reported.  The patient had COVID-19 again on 11-APR-2023 and on an unknown date in 2023. No treatment information was reported.   This case was linked to US-MODERNATX, INC.-MOD-2023-753450 (Patient Link).",UNK,2
83,2729334,COVID19-2,"Covid-19; This spontaneous case was reported by a patient and describes the occurrence of COVID-19 (Covid-19) in a 72-year-old male patient who received mRNA-1273 BIVALENT .222 (MODERNA COVID-19 VACCINE, BIVALENT (ORIGINAL AND OMICRON BA.4/BA.5)) (batch no. AS7144B) for COVID-19 prophylaxis.     Previously administered products included for COVID-19 prophylaxis: Moderna covid-19 vaccine (First dose lot number 011L20A) on 08-Jan-2021, Moderna covid-19 vaccine (Second dose lot number 043L20A) on 05-Feb-2021 and Moderna covid-19 vaccine (Third dose lot number 035C21A) since 06-Oct-2021. Past adverse reactions to the above products included No adverse event with Moderna covid-19 vaccine, Moderna covid-19 vaccine and Moderna covid-19 vaccine.    On 10-Sep-2022, the patient received fourth dose of mRNA-1273 BIVALENT .222 (MODERNA COVID-19 VACCINE, BIVALENT (ORIGINAL AND OMICRON BA.4/BA.5)) (unknown route) 1 dosage form. On 02-Oct-2023,  after starting mRNA-1273 BIVALENT .222 (MODERNA COVID-19 VACCINE, BIVALENT (ORIGINAL AND OMICRON BA.4/BA.5)), the patient experienced COVID-19 (Covid-19). At the time of the report, COVID-19 (Covid-19) outcome was unknown.            No concomitant medication was reported.  No treatment information was reported.   This case was linked to MOD-2023-755360 (Patient Link).",UNK,2
84,2729335,COVID19,"vertigo; pre syncope; This spontaneous case was reported by a consumer and describes the occurrence of VERTIGO (vertigo) and PRESYNCOPE (pre syncope) in a 66-year-old female patient who received mRNA-1273.815 (SPIKEVAX 2023-2024) for COVID-19 prophylaxis.     No Medical History information was reported.    On 07-Nov-2023, the patient received dose of mRNA-1273.815 (SPIKEVAX 2023-2024) (unknown route) 1 dosage form. On 05-Dec-2023, the patient experienced VERTIGO (vertigo) (seriousness criterion medically significant) and PRESYNCOPE (pre syncope) (seriousness criterion medically significant). At the time of the report, VERTIGO (vertigo) and PRESYNCOPE (pre syncope) had resolved.        The action taken with mRNA-1273.815 (SPIKEVAX 2023-2024) (Unknown) was unknown.       Concomitant product use was not provided by the reporter.  Treatment information was not provided.   Company Comment This spontaneous case concerns a 66 year-old female patient with no medical history  reported who experienced the unexpected seriousness criteria medically significant events of vertigo and presyncope  28 days after a dose of mRNA-1273.815 vaccine. Concomitant medication nor Treatment details were reported. At the time of the report, the events had resolved.    The benefit-risk relationship of mRNA-1273.815 vaccine is not affected by this report.   Reporter did not allow further contact",UNK,2
85,2729336,COVID19,"I have tested positive for HPV after getting vaccine; This spontaneous case was reported by a patient and describes the occurrence of PAPILLOMA VIRAL INFECTION (I have tested positive for HPV after getting vaccine) in a 46-year-old female patient who received mRNA-1273.815 (SPIKEVAX 2023-2024) for COVID-19 prophylaxis.     No Medical History information was reported.    On an unknown date, the patient received third dose of mRNA-1273.815 (SPIKEVAX 2023-2024) (unknown route) 1 dosage form. On an unknown date, the patient experienced PAPILLOMA VIRAL INFECTION (I have tested positive for HPV after getting vaccine). At the time of the report, PAPILLOMA VIRAL INFECTION (I have tested positive for HPV after getting vaccine) had not resolved.      DIAGNOSTIC RESULTS (normal ranges are provided in parenthesis if available): On an unknown date, Human papilloma virus test: Positive. On an unknown date, Mammogram: abnormal mammograms.     For mRNA-1273.815 (SPIKEVAX 2023-2024) (Unknown), the reporter did not provide any causality assessments.   No Concomitant medications provided by the reporter. Patient received 1st and second dose. The patient had no concomitant disease or risk factor. No treatment medications provided by the reporter.",UNK,2
86,2729337,COVID19,"The patient explained, the problem is I'm literally losing all the connective tissues and stuff that makes my arm round. It's all over my body, my shoulder, and it's on my right leg. It's going to the other side of my body; This spontaneous case was reported by a patient and describes the occurrence of CONNECTIVE TISSUE DISORDER (The patient explained, the problem is I'm literally losing all the connective tissues and stuff that makes my arm round. It's all over my body, my shoulder, and it's on my right leg. It's going to the other side of my body) in a 48-year-old male patient who received mRNA-1273 (Moderna COVID-19 Vaccine) (batch nos. 033F21A, 026B21A and 002C21A) for COVID-19 prophylaxis.     No Medical History information was reported.   On 29-Mar-2021, the patient received first dose of mRNA-1273 (Moderna COVID-19 Vaccine) (unknown route) 1 dosage form. On 26-Apr-2021, received second dose of mRNA-1273 (Moderna COVID-19 Vaccine) (unknown route) dosage was changed to 1 dosage form. On 16-Nov-2021, received third dose of mRNA-1273 (Moderna COVID-19 Vaccine) (unknown route) dosage was changed to 1 dosage form. On an unknown date, the patient experienced CONNECTIVE TISSUE DISORDER (The patient explained, the problem is I'm literally losing all the connective tissues and stuff that makes my arm round. It's all over my body, my shoulder, and it's on my right leg. It's going to the other side of my body) (seriousness criterion medically significant). At the time of the report, CONNECTIVE TISSUE DISORDER (The patient explained, the problem is I'm literally losing all the connective tissues and stuff that makes my arm round. It's all over my body, my shoulder, and it's on my right leg. It's going to the other side of my body) had not resolved.        For mRNA-1273 (Moderna COVID-19 Vaccine) (Unknown), the reporter did not provide any causality assessments.   The concomitant medication was not reported by reporter. The patient stated that a couple of months after that it started with him eating at the table and he would rest his wrist on the table and could feel bones in his wrist, he saw veins in forearm. Patient stated that he received the initial doses in 2021, but believes it is from the Moderna Booster he received 16-Nov-2021. It started with him being able to feel bones that he couldn't feel before because there was protection. All the protective tissue that covers his muscle was gone. He can saw bone and vein and there was no fat over it. It was moving all over his body. He was down to like his bone in the middle of my right forearm. He started being able to feel bones in his right arm that he could never felt before, the bones were sticking out. Something within him was eating his fat. Patient stated, he can feel bones in his arms, and all connective tissues in gone, his arm was no longer round, his right shoulder was like a cage. His right chest, to his shoulder was like a cage and you can rest your hand in there. This thing was relentlessly keeping on going. His right leg was duplicating the same symptoms as his right arm. His right thigh also was starting to dimple in both sides of his leg. Patient was totally healthy and in the best shape of his life, He lost some strength right side, and lost tissue. Patient stated, he could print probably 200 pages of different tests. He had taken. Patient had every blood test under the sun, Lyme-disease, Lupus like anything and everything. He was not going to lie. Rheumatologist, neurologist, and Cancer doctor and they just look at his at this and don't know what he had; he had been literally lost half of the right arm. He was running out of time. He watched his body eat itself for a year and a half. The patient stated that he's had the Pfizer in 26-Sep-2022- Pfizer, GH9693, but all the problems were already on him like.  Company comment: This spontaneous case concerns a 48  year  old male patient, with no medical history reported, who received three doses of Moderna COVID-19 Vaccine (mRNA  1273) and experienced the unexpected, serious (medically significant) event of connective tissue disorder. Latency cannot be properly assessed since exact onset date was not provided. The report stated that months after receiving the booster vaccine he ""has been losing body tissue all over his body"". Rheumatologist, Neurologist, and Cancer doctors were consulted and ""200 pages of different laboratory tests"" were performed, included for Lupus like syndrome or Lyme disease; however, no diagnosis was provided. The patient explained that his problem is that he is ""losing all the connective tissues It's all over the body"". The outcome of the event was not recovered. No further clinical details were provided for medical reviewing. The Benefit-risk relationship of Moderna COVID-19 Vaccine (mRNA  1273) is not affected by this report.",OTH,2
87,2729338,COVID19,"Patient started having Urticaria; This spontaneous case was reported by a patient and describes the occurrence of URTICARIA (Patient started having Urticaria) in a 39-year-old male patient who received mRNA-1273.815 (SPIKEVAX 2023-2024) (batch no. AU3914B) for COVID-19 prophylaxis.     Previously administered products included for COVID-19 prophylaxis: Moderna COVID-19 Vaccine (Dose-1, Lot 029L20A) on 19-Jan-2021, Moderna COVID-19 Vaccine (Dose-2, Lot 024M20A) on 19-Jan-2021, Moderna COVID-19 Vaccine (Dose-2, Lot 024M20A) on 16-Feb-2021, Moderna COVID-19 Vaccine (Dose 3 and Lot 022321A) on 23-Nov-2021. Past adverse reactions to the above products included No adverse event with Moderna COVID-19 Vaccine, Moderna COVID-19 Vaccine, Moderna COVID-19 Vaccine and Moderna COVID-19 Vaccine.    On 06-Dec-2023, the patient received fourth dose of mRNA-1273.815 (SPIKEVAX 2023-2024) (unknown route) 1 dosage form. On 22-Dec-2023,  after starting mRNA-1273.815 (SPIKEVAX 2023-2024), the patient experienced URTICARIA (Patient started having Urticaria). The patient was treated with CETIRIZINE HYDROCHLORIDE (ZYRTEC ALLERGY) for Urticaria, at a dose of PO Twice a day and TRIAMCINOLONE for Urticaria, at a dose of 0.1% Cream. At the time of the report, URTICARIA (Patient started having Urticaria) had not resolved.            The concomitant medication was not reported by reporter. It was reported that patient brakes out on random parts of the body. It disappears from that part and pops up in another area. Patient visited a dermatologist and was diagnosed with urticaria on 26-DEC-2023.",UNK,2
88,2729339,COVID19-2,"covid 19; This spontaneous case was reported by a patient and describes the occurrence of COVID-19 (covid 19) in an adult female patient who received mRNA-1273 BIVALENT .222 (MODERNA COVID-19 VACCINE, BIVALENT (ORIGINAL AND OMICRON BA.4/BA.5)) (batch no. NASK) for COVID-19 prophylaxis.     No Medical History information was reported.    In November 2022, the patient received dose of mRNA-1273 BIVALENT .222 (MODERNA COVID-19 VACCINE, BIVALENT (ORIGINAL AND OMICRON BA.4/BA.5)) (unknown route) 1 dosage form. On 05-Dec-2023, the patient experienced COVID-19 (covid 19). On 10-Dec-2023, COVID-19 (covid 19) had resolved.      DIAGNOSTIC RESULTS (normal ranges are provided in parenthesis if available): On 05-Dec-2023, SARS-CoV-2 test: Positive. On 10-Dec-2023, SARS-CoV-2 test: Negative.         She had all her previous vaccinations as Moderna COVID-19 vaccines.  The concomitant medication was not reported by reporter.   The treatment medication was not reported by reporter.   Reporter did not allow further contact",UNK,2
89,2729340,COVID19,"Elevated tachycardic heart rate; This spontaneous case was reported by a patient and describes the occurrence of HEART RATE INCREASED (Elevated tachycardic heart rate) in a 72-year-old male patient who received mRNA-1273.815 (SPIKEVAX 2023-2024) for COVID-19 prophylaxis.     Previously administered products included for COVID-19 prophylaxis: Moderna covid-19 vaccine (First dose lot number 011L20A) on 08-Jan-2021, Moderna covid-19 vaccine (Second dose lot number 043L20A) on 05-Feb-2021 and Moderna covid-19 vaccine (Third dose lot number 035C21A) on 06-Oct-2021; for Product used for unknown indication: Flu vaccine in 2023. Past adverse reactions to the above products included No adverse event with Flu vaccine, Moderna covid-19 vaccine, Moderna covid-19 vaccine and Moderna covid-19 vaccine.    In December 2023, the patient received fifth dose of mRNA-1273.815 (SPIKEVAX 2023-2024) (unknown route) 1 dosage form. In December 2023, the patient experienced HEART RATE INCREASED (Elevated tachycardic heart rate). At the time of the report, HEART RATE INCREASED (Elevated tachycardic heart rate) had not resolved.      DIAGNOSTIC RESULTS (normal ranges are provided in parenthesis if available): In December 2023, Heart rate: Increased.         Patient did receive covid-19 vaccination on OCT-2023.  No concomitant medication was reported.  Patient was experiencing an elevated tachycardic heart rate. It started 30 hours ago and has continued. Patient didn't take any medications or treatments. He did not drink as much coffee. No treatment information was reported.   This case was linked to MOD-2023-755361 (Patient Link).",UNK,2
90,2729344,COVID19,"tested positive. Her only symptom is post-nasal drip. However a home test kit tested her positive for COVID-19.; tested positive. Her only symptom is post-nasal drip. However a home test kit tested her positive for COVID-19.; The initial case was missing the following minimum criteria: Other-Non Pfizer product. Upon receipt of follow-up information on 19Dec2023, this case now contains all required information to be considered valid.  This is a spontaneous report received from a Consumer or other non HCP.  A 56-year-old female patient (not pregnant) received BNT162b2 omi xbb.1.5 (COMIRNATY (2023-2024 FORMULA)), on 30Sep2023 as dose 1, single (Batch/Lot number: unknown) at the age of 56 years intramuscular, in right arm for covid-19 immunisation. The patient's relevant medical history included: ""High blood pressure"" (unspecified if ongoing). Concomitant medication(s) included: NIFEDIPINE. Vaccination history included: Covid-19 vaccine (Dose 1; unknown manufacturer, ), for Covid-19 immunization; Covid-19 vaccine (Dose 2; unknown manufacturer, ), for Covid-19 immunization; Covid-19 vaccine (Dose 3 (booster); unknown manufacturer, ), for Covid-19 immunization; Covid-19 vaccine (Dose 4 (booster); unknown manufacturer, ), for COVID-19 immunization. The following information was reported: VACCINATION FAILURE (medically significant), COVID-19 (medically significant) all with onset 19Dec2023, outcome ""recovering"" and all described as ""tested positive. Her only symptom is post-nasal drip. However a home test kit tested her positive for COVID-19."". The patient underwent the following laboratory tests and procedures: SARS-CoV-2 test: (19Dec2023) Positive, notes: Nasal Swab. Therapeutic measures were not taken as a result of vaccination failure, covid-19.  Clinical course: They tested (name withheld) after my daughter tested positive. Her only symptom is post-nasal drip. However, a home test kit tested her positive for COVID-19. No covid prior vaccination. It was Comirnaty that was administered. (name withheld) and reporter and also both received Comirnaty. Reporter was confused on the reporting form and did not understand that Comirnaty is not the same thing as ""bivalent"".   The information on the batch/lot number for BNT162b2 omi xbb.1.5 will be requested and submitted if and when received.",PHM,2
91,2729346,COVID19,"feel quite confused (brain fog); feel quite confused (brain fog); unsteady on his feet; This is a spontaneous report received from a Consumer or other non HCP and a Nurse.  A 79-year-old male patient received rsv vaccine prot.subunit pref 2v (ABRYSVO), on 09Nov2023 as dose number unknown, single (Batch/Lot number: unknown) at the age of 79 years for immunisation; elasomeran (MODERNA COVID-19 VACCINE), on 09Nov2023 as dose number unknown, single) for covid-19 immunisation. The patient had no relevant medical history. There were no concomitant medications. The following information was reported: BRAIN FOG (non-serious), CONFUSIONAL STATE (non-serious) all with onset 15Nov2023, outcome ""recovering"" and all described as ""feel quite confused (brain fog)""; GAIT DISTURBANCE (non-serious) with onset 15Nov2023, outcome ""recovering"", described as ""unsteady on his feet"". The events ""feel quite confused (brain fog)"" and ""unsteady on his feet"" required physician office visit. Therapeutic measures were not taken as a result of brain fog, confusional state, gait disturbance.  Additional information: Caller stated that he called about Abrysvo (RSV), a Pfizer product. On 09Nov2023 he went into (name withheld) and they did his Covid and RSV shots and within a day or so he was starting to feel quite confused (brain fog) and  unsteady on his feet that continued until last Thursday (14Dec2023). Now he feels fine. Had Moderna Covid shot. Had talked with the nurse at the (name withheld) before they called Pfizer and she told them that the Covid shot was Moderna. Later clarified that (name withheld) was for (name withheld). Attempted to capture outcome: stated that there are still some iffy spots but much better.  The information on the batch/lot number for rsv vaccine prot.subunit pref 2v will be requested and submitted if and when received.",UNK,2
92,2729349,COVID19,"pain in the injection site that he developed after getting the vaccine; arm hurts / arm pain / ever since then his arm had been giving him pain; he could hardly at times pick up a cup of coffee and in different positions it hurt worse than others; This is a spontaneous report received from a Consumer or other non HCP from medical information team.  An 82-year-old male patient received BNT162b2 omi xbb.1.5 (COMIRNATY (2023-2024 FORMULA)), on 10Nov2023 at 12:00 as dose 1, single (Batch/Lot number: unknown) at the age of 81 years, in right arm for covid-19 immunisation. The patient's relevant medical history included: ""Blood pressure high"" (unspecified if ongoing). The patient's concomitant medications were not reported. The following information was reported: PAIN IN EXTREMITY (non-serious) with onset 2023, outcome ""unknown"", described as ""arm hurts / arm pain / ever since then his arm had been giving him pain""; MOBILITY DECREASED (non-serious) with onset 2023, outcome ""unknown"", described as ""he could hardly at times pick up a cup of coffee and in different positions it hurt worse than others""; VACCINATION SITE PAIN (non-serious) with onset 2023, outcome ""not recovered"", described as ""pain in the injection site that he developed after getting the vaccine"". Therapeutic measures were not taken as a result of pain in extremity. Additional information: the patient had been having pain in his arm near where the shot was put in and it had gotten so bad he could hardly at times pick up a cup of coffee and in different positions it hurt worse than others and if he did not move it, it did not hurt. No other vaccines were administered in 4 weeks  Batch/lot number is not provided, and it cannot be obtained.",PHM,2
93,2729351,COVID19,"COVID 19 Treatment; COVID 19 Treatment; This is a spontaneous report received from a Consumer or other non HCP from product quality group.  A 68-year-old female patient (not pregnant) received BNT162b2 omi xbb.1.5 (COMIRNATY (2023-2024 FORMULA)), on 25Sep2023 at 14:00 as dose 1, single (Lot number: HD9835) at the age of 68 years, in left arm for covid-19 immunisation. The patient's relevant medical history included: ""High Blood Pressure"" (unspecified if ongoing); ""Osteoporosis"" (unspecified if ongoing); ""Sulfonamide allergy"" (unspecified if ongoing), notes: Sulfa drugs, like the ones given for UTIs. Concomitant medication(s) included: RALOXIFENE, start date: Feb2021, stop date: Dec2023; LOSARTAN, start date: May2023, stop date: Dec2023. Vaccination history included: BNT162b2 (DOSE 1, SINGLE, Lot number=EL1283, vaccine location=Left arm), administration date: 20Jan2021, when the patient was 66-year-old, for Covid-19 immunization; BNT162b2 (DOSE 2, SINGLE, Lot number=EN5318, vaccine location=Left arm), administration date: 10Feb2021, when the patient was 66-year-old, for Covid-19 immunization; BNT162b2 (DOSE 3 (BOOSTER), SINGLE, Lot number=FD8448, vaccine location=Left arm), administration date: 26Aug2021, when the patient was 66-year-old, for Covid-19 immunization; BNT162b2 (DOSE 4 (BOOSTER), SINGLE, Lot number=FK9893, vaccine location=Left arm), administration date: 11Apr2022, when the patient was 67-year-old, for Covid-19 immunization; BNT162b2;BNT162b2 omi ba.4-5 (DOSE 5 (BOOSTER), SINGLE, Lot number=GH9702, vaccine location=Left arm), administration date: 16Sep2022, when the patient was 67-year-old, for Covid-19 immunization. The following information was reported: VACCINATION FAILURE (medically significant), COVID-19 (medically significant) all with onset Dec2023, outcome ""recovered"" (Dec2023) and all described as ""COVID 19 Treatment"". The patient underwent the following laboratory tests and procedures: SARS-CoV-2 test: Positive, notes: Tested positive for 9 days; negative, notes: negative for 2 day. Therapeutic measures were taken as a result of vaccination failure, covid-19 with Paxlovid from 17Dec2023 to 19Dec2023.",UNK,2
94,2729352,COVID19,"sore throat/throat is super sore; her ears hurt.; was coughing; In the middle of the night, aches/Body aches and was pretty bad, lasted for a probably about 30 hours; shivering/extreme chills; she developed pain in the injection site/arm started hurting; This is a spontaneous report received from a Consumer or other non HCP from medical information team.  A 55-year-old female patient received BNT162b2 omi xbb.1.5 (COMIRNATY (2023-2024 FORMULA)), on 23Dec2023 as dose 1, single (Batch/Lot number: unknown) at the age of 55 years, in arm for covid-19 immunisation. The patient's relevant medical history and concomitant medications were not reported. Caller stated that on the evening on 23Dec2023 when she received the vaccine everything was fine then her arm started hurting, she developed pain in the injection site, then on Sunday (24Dec2023), the middle of the night, she started getting shivering/ extreme chills and aches/ body aches and was pretty bad and it was crazy and that lasted for a probably about 30 hours and she took Tylenol (Intent: Treatment) which seems to help and made it just so she was felt like she was not going to die and but then yesterday on 26Dec2023 she developed sore throat and her ears hurt. Caller stated she started coughing it sounded, it felt like it was from her lungs, her throat was not sore. Now she was kind of stop coughing and her throat was super sore. Caller added that the coughing came first then the sore throat comes next. Therapeutic measures were taken as a result of pain, vaccination site pain. The outcome of event Pain, Chills, Vaccination site pain and Cough was recovered in Dec2023. The outcome of the other events was unknown.  The information on the batch/lot number for BNT162b2 omi xbb.1.5 will be requested and submitted if and when received.",UNK,2
95,2729353,COVID19,"I developed Neuropathy; This is a spontaneous report received from a Consumer or other non HCP from medical information team, Program ID.  A patient (age and gender not provided) received BNT162b2 omi xbb.1.5 (COMIRNATY (2023-2024 FORMULA)), in 2023 as dose 1, single (Batch/Lot number: unknown) for covid-19 immunisation. The patient's relevant medical history and concomitant medications were not reported. Vaccination history included: Bnt162b2 nos (DOSE NUMBER UNKNOWN, SINGLE), for COVID-19 immunisation. The following information was reported: NEUROPATHY PERIPHERAL (medically significant) with onset 2023, outcome ""unknown"", described as ""I developed Neuropathy"". The event ""i developed neuropathy"" required physician office visit. Clinical course: After getting the first two shots of the Pfizer Covid vaccine patient developed Neuropathy. Patient's Neurologist told patient that patient wasn't the only one.   The information on the batch/lot number for BNT162b2 omi xbb.1.5 will be requested and submitted if and when received.",UNK,2
96,2729354,COVID19,"COVID 19 Treatment; COVID 19 Treatment; This is a spontaneous report received from a Consumer or other non HCP.  A 59-year-old female patient (not pregnant) received BNT162b2 omi xbb.1.5 (COMIRNATY (2023-2024 FORMULA)), on 13Oct2023 at 19:00 as dose 1, single (Batch/Lot number: unknown) at the age of 59 years, in left arm for covid-19 immunisation. The patient's relevant medical history included: ""High BMI"" (unspecified if ongoing), notes: Over 50, high BMI; ""strawberries"" (unspecified if ongoing). There were no concomitant medications. Past drug history included: Codeine, reaction(s): ""Drug allergies: Codeine""; Prednisone, reaction(s): ""Drug allergies: Prednisone"". Vaccination history included: BNT162b2 ({Vaccine brand=Pfizer / Biontech, Vaccine brand unknown=False, Vaccine administration date=17Nov2022, Vaccine administration time=05:00 PM, Vaccine lot unknown=true, Vaccine lot unknown reason=unable to locate or Read the details, Vaccine dose number unknown=true, Vaccine dose number unknown reason=unknown, Vaccine location=left arm}]), administration date: 17Nov2022, when the patient was 58-year-old, for COVID-19 Immunization. The following information was reported: VACCINATION FAILURE (medically significant), COVID-19 (medically significant) all with onset Dec2023, outcome ""unknown"" and all described as ""COVID 19 Treatment"". The patient underwent the following laboratory tests and procedures: Body mass index: (unspecified date) Over 50, high BMI, notes: Over 50, high BMI; SARS-CoV-2 test: (2023) negative, notes: previously had tested negative on two consecutive tests 48 hours aparrt.; (2023) positive, notes: Tested positive again on at home test. Therapeutic measures were taken as a result of vaccination failure, covid-19.  Clinical course: The patient received anti viral covid 19 treatment with paxlovid from 15Dec2023 to 20Dec2023.   The information on the batch/lot number for BNT162b2 omi xbb.1.5 will be requested and submitted if and when received.",UNK,2
97,2729355,COVID19,"COVID 19; COVID 19; Dose 2/Dose 1:Pfizer;Dose 5/Dose 4/Dose 3:Moderna; This is a spontaneous report received from a Consumer or other non HCP.  A 42-year-old female patient (not pregnant) received BNT162b2 (BNT162B2), on 19Apr2021 as dose 1, single (Batch/Lot number: unknown), in right arm and on 09May2021 as dose 2, single (Batch/Lot number: unknown) at the age of 40 years, in right arm for covid-19 immunisation; elasomeran (MODERNA COVID-19 VACCINE), on 04Oct2021 as dose 3 (booster), single), in right arm, on 03Oct2022 at 12:00 as dose 4 (booster), single), in right arm and on 20Oct2023 at 17:30 as dose 5 (booster), single), in right arm for covid-19 immunisation. The patient's relevant medical history included: ""Hypothyroidism"" (unspecified if ongoing); ""Penicillin allergy"" (unspecified if ongoing); ""sulfa-based antibiotics allergy"" (unspecified if ongoing). The patient's concomitant medications were not reported. The following information was reported: INTERCHANGE OF VACCINE PRODUCTS (medically significant) with onset 04Oct2021, outcome ""recovered"", described as ""Dose 2/Dose 1:Pfizer;Dose 5/Dose 4/Dose 3:Moderna""; DRUG INEFFECTIVE (medically significant), COVID-19 (medically significant) all with onset 02Dec2023, outcome ""recovered"" (10Dec2023) and all described as ""COVID 19"". The patient underwent the following laboratory tests and procedures: SARS-CoV-2 test: (04Dec2023) Positive; (10Dec2023) Negative. Therapeutic measures were taken as a result of drug ineffective, covid-19.  Clinical course: Initial covid symptoms began on 02Dec2023. Tested positive on 04Dec2023. Started Paxlovid midday on 04Dec2023 and completed the full 5 days in the evening on 08Dec2023. Symptoms were flu-like and not serious enough for hospital visit. Began to wane on 09Dec2023 and were mostly clear on 10Dec2023. Negative test on 10Dec2023.  The information on the batch/lot number for BNT162b2 will be requested and submitted if and when received.",UNK,2
98,2729356,COVID19,"COVID 19; COVID 19; This is a spontaneous report received from a Consumer or other non HCP.  A 29-year-old male patient received BNT162b2, BNT162b2 omi ba.4-5 (BNT162B2, BNT162B2 OMI BA.4-5), on 18Sep2022 as dose 4 (booster), single (Lot number: GH9694) at the age of 28 years, in left arm for covid-19 immunisation; COVID-19 vaccine (COVID-19 VACCINE), as dose 1, single), as dose 2, single) and as dose 3 (booster), single) for covid-19 immunisation. The patient's relevant medical history and concomitant medications were not reported. The following information was reported: DRUG INEFFECTIVE (medically significant), COVID-19 (medically significant) all with onset Dec2023, outcome ""recovering"" and all described as ""COVID 19"". The patient underwent the following laboratory tests and procedures: SARS-CoV-2 test: (27Dec2023) Negative; (27Dec2023) Negative; (27Dec2023) Positive. Therapeutic measures were taken as a result of drug ineffective, covid-19.  Clinical course: The patient received Paxlovid treatment from 21Dec2023 to 25Dec2023. Two days after finishing Paxlovid he took 3 COVID tests with 2 being negative and 1 being positive. He was unsure if this was just a false positive or not but he was concerned.",UNK,2
99,2729356,COVID19,"COVID 19; COVID 19; This is a spontaneous report received from a Consumer or other non HCP.  A 29-year-old male patient received BNT162b2, BNT162b2 omi ba.4-5 (BNT162B2, BNT162B2 OMI BA.4-5), on 18Sep2022 as dose 4 (booster), single (Lot number: GH9694) at the age of 28 years, in left arm for covid-19 immunisation; COVID-19 vaccine (COVID-19 VACCINE), as dose 1, single), as dose 2, single) and as dose 3 (booster), single) for covid-19 immunisation. The patient's relevant medical history and concomitant medications were not reported. The following information was reported: DRUG INEFFECTIVE (medically significant), COVID-19 (medically significant) all with onset Dec2023, outcome ""recovering"" and all described as ""COVID 19"". The patient underwent the following laboratory tests and procedures: SARS-CoV-2 test: (27Dec2023) Negative; (27Dec2023) Negative; (27Dec2023) Positive. Therapeutic measures were taken as a result of drug ineffective, covid-19.  Clinical course: The patient received Paxlovid treatment from 21Dec2023 to 25Dec2023. Two days after finishing Paxlovid he took 3 COVID tests with 2 being negative and 1 being positive. He was unsure if this was just a false positive or not but he was concerned.",UNK,2
100,2729357,COVID19,"COVID 19; COVID 19; This is a spontaneous report received from a Nurse.  A 38-year-old female patient (not pregnant) received BNT162b2, BNT162b2 omi ba.4-5 (BNT162B2, BNT162B2 OMI BA.4-5), on 01Oct2022 as dose 4 (booster), single (Batch/Lot number: unknown) at the age of 37 years, in left arm for covid-19 immunisation; COVID-19 vaccine (COVID-19 VACCINE), as dose 1, single), as dose 2, single) and as dose 3 (booster), single). The patient's relevant medical history included: ""Sarcoidosis"" (unspecified if ongoing); ""Gluten intolerance"" (unspecified if ongoing); ""Dairy intolerance"" (unspecified if ongoing). The patient's concomitant medications were not reported. The following information was reported: DRUG INEFFECTIVE (medically significant), COVID-19 (medically significant) all with onset 2023, outcome ""recovered"" (Dec2023) and all described as ""COVID 19"". The patient underwent the following laboratory tests and procedures: SARS-CoV-2 test: (Dec2023) Negative; (Dec2023) Negative. Therapeutic measures were taken as a result of drug ineffective, covid-19.  Clinical course: Paxlovid treatment start date 18Dec2023, stop date 22Dec2023 for COVID-19. Tested negative at days 6, 7 and positive on days 8, 9 (today). Runny nose but otherwise no additional symptoms. Known allergies: Gluten and dairy intolerance not true allergies.  The information on the batch/lot number for BNT162b2, BNT162b2 omi ba.4-5 will be requested and submitted if and when received.; Sender's Comments: The efficacy of a drug varies from patient to patient and can be affected by varied factors; however, a contributory role of the suspect product to the reported Drug ineffective /COVID-19 cannot be ruled out.",UNK,2

"""

In [ ]:
prompt = f"""The following is a section of VAERS DATASET:
    {document}

    You are medical symptom extractor.
    Extract all the medical symptoms from SYMPTOM_TEXT column and return it as a structured table.include all the given rows and columns along with the newly added column as extracted_symptoms. be consistent with symptoms extracted for example, head hurts, pain in the head, etc should be headache or high temparature, high fever, hot to touch etc could be just fever, maintain a dictionary of symptom words extracted and use medical synonyms return only the VAERS_ID and extracted_symtoms. 
    Provide the table as JSON object."""

In [49]:
response = model.generate_content(prompt)
print(response.text)

```json
[
  {
    "VAERS_ID": 2728994,
    "extracted_symptoms": ["muscle pain", "arm pain", "pain"]
  },
  {
    "VAERS_ID": 2729004,
    "extracted_symptoms": ["seizures", "micro-seizures", "speech arrest", "eye rolling", "arm stiffness", "encephalitis"]
  },
  {
    "VAERS_ID": 2729006,
    "extracted_symptoms": ["endocarditis", "fever", "chills", "lung infection"]
  },
  {
    "VAERS_ID": 2729006,
    "extracted_symptoms": ["endocarditis", "fever", "chills", "lung infection"]
  },
  {
    "VAERS_ID": 2729006,
    "extracted_symptoms": ["endocarditis", "fever", "chills", "lung infection"]
  },
  {
    "VAERS_ID": 2729010,
    "extracted_symptoms": ["fever", "aches", "pain", "fatigue", "chills", "night sweats", "diarrhea", "loss of appetite"]
  },
  {
    "VAERS_ID": 2729014,
    "extracted_symptoms": ["shoulder pain"]
  },
  {
    "VAERS_ID": 2729015,
    "extracted_symptoms": ["hives"]
  },
  {
    "VAERS_ID": 2729016,
    "extracted_symptoms": ["lumps"]
  },
  {
    "VAERS_ID": 27

In [88]:
import pandas as pd
df=pd.read_csv('symptoms_covid.csv')


In [89]:
df

,Unnamed: 0,VAERS_ID,VAX_TYPE,SYMPTOM_TEXT,V_ADMINBY,FORM_VERS
0,0,2728994,COVID19,muscle at the injection site is sore; He canno...,PHM,2
1,1,2729004,COVID19,Micro-seizures starting approximately 2 weeks ...,OTH,2
2,2,2729006,COVID19,"On September 14, 2023, I was admitted to Hospi...",PHM,2
3,3,2729006,COVID19,"On September 14, 2023, I was admitted to Hospi...",PHM,2
4,4,2729006,COVID19,"On September 14, 2023, I was admitted to Hospi...",PHM,2
...,...,...,...,...,...,...
17420,17551,2802236,COVID19,Covid Pfizer Vaccine was stored in the freezer...,UNK,2
17421,17552,2802237,COVID19,Covid Pfizer Vaccine was stored in the freezer...,UNK,2
17422,17553,2802238,COVID19,Covid Pfizer Vaccine was stored in the freezer...,UNK,2
17423,17554,2802239,COVID19,Covid Pfizer Vaccine was stored in the freezer...,UNK,2


In [90]:
df = df.drop(df.columns[:1], axis=1)
# Set 'VAERS_ID' as the index
df.set_index('VAERS_ID', inplace=True)

In [92]:
df.to_csv('data_for_llm.csv')

In [93]:
with open('data_for_llm.txt', 'r') as f:
    data = f.readlines()


In [94]:
len(data)

17426

In [ ]:
import re, json
k = 50
i=0
while i < len(data):
   
        document = data[i:i+k]
        prompt = F"""The following is a section of VAERS DATASET:
        {document}

        You are medical symptom extractor.
        Extract all the medical symptoms from SYMPTOM_TEXT column and return it as a structured table.include all the given rows and columns along with the newly added column as extracted_symptoms. be consistent with symptoms extracted for example, head hurts, pain in the head, etc should be headache or high temparature, high fever, hot to touch etc could be just fever, maintain a dictionary of symptom words extracted and use medical synonyms return only the VAERS_ID and extracted_symtoms. 
        YOUR OUTPUT SHOULD LOOK LIKE THIS:
        <CURLY BRACKET>"VAERS_ID":<NUMERIC ID> , "extracted_symptoms": ["SYMTOM1", "SYMPTOM2" ]<CURLY BRACKET>
        Provide the table as JSON object."""

        response = model.generate_content(prompt)
        print(response.text)
        response_text = response.text
        if response_text.startswith("```json"):
            response_text = response_text[7:]  # Remove the prefix '```json\n'
        if response_text.endswith("```"):
            response_text = response_text[:-3]  # Remove the suffix '```'

    # Strip any remaining leading/trailing whitespace or newlines
        response_text = response_text.strip()
        response_text = response_text.strip()
        json_objects = re.findall(r'\{.*?\}', response_text)
        json_strings = [json.dumps(obj) for obj in json_objects]
        with open('output.txt', 'a') as f:
            for string in json_strings:
                f.write(string + '\n')

        i = i+k
        


    

    

```json
[
  {"VAERS_ID": 2728994, "extracted_symptoms": ["muscle pain", "arm soreness", "pain", "activities of daily living impaired"]},
  {"VAERS_ID": 2729004, "extracted_symptoms": ["micro-seizures", "seizures", "arrest in speech", "eye rolling", "arm stiffness", "seizure", "encephalitis"]},
  {"VAERS_ID": 2729006, "extracted_symptoms": ["endocarditis", "fever", "chills", "lung infection"]},
  {"VAERS_ID": 2729006, "extracted_symptoms": ["endocarditis", "fever", "chills", "lung infection"]},
  {"VAERS_ID": 2729006, "extracted_symptoms": ["endocarditis", "fever", "chills", "lung infection"]},
  {"VAERS_ID": 2729010, "extracted_symptoms": ["fever", "aches and pains", "fatigue", "chills", "night sweats", "diarrhea", "loss of appetite"]},
  {"VAERS_ID": 2729014, "extracted_symptoms": ["shoulder pain"]},
  {"VAERS_ID": 2729015, "extracted_symptoms": ["hives"]},
  {"VAERS_ID": 2729016, "extracted_symptoms": ["lumps"]},
  {"VAERS_ID": 2729017, "extracted_symptoms": ["brain fog", "slowed thi

In [78]:
import re
json_objects = re.findall(r'\{.*?\}', response_text)

In [79]:
json_objects

['{"VAERS_ID": 2728994, "extracted_symptoms": ["muscle pain", "arm soreness", "pain", "activities of daily living impaired"]}',
 '{"VAERS_ID": 2729004, "extracted_symptoms": ["micro-seizures", "seizures", "arrest in speech", "eye rolling", "arm stiffness", "encephalitis"]}',
 '{"VAERS_ID": 2729006, "extracted_symptoms": ["endocarditis", "fever", "chills", "lung infection"]}',
 '{"VAERS_ID": 2729006, "extracted_symptoms": ["endocarditis", "fever", "chills", "lung infection"]}',
 '{"VAERS_ID": 2729006, "extracted_symptoms": ["endocarditis", "fever", "chills", "lung infection"]}',
 '{"VAERS_ID": 2729010, "extracted_symptoms": ["fever", "aches and pains", "fatigue", "chills", "night sweats", "diarrhea", "loss of appetite"]}',
 '{"VAERS_ID": 2729014, "extracted_symptoms": ["shoulder pain"]}',
 '{"VAERS_ID": 2729015, "extracted_symptoms": ["hives"]}',
 '{"VAERS_ID": 2729016, "extracted_symptoms": ["lumps"]}',
 '{"VAERS_ID": 2729017, "extracted_symptoms": ["brain fog", "slowed thinking"]}',
 